## 1. 데이터 전처리

runpod에서 a100 GPU 1개 sxm 대여해주시고 container disk는 50기가 바이트

In [1]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

빠른 학습을 위해 학습 데이터와 테스트 데이터를 2:8 비율로 분할합니다. 이 값을 변경하고자 하는 분은 test_ratio의 값을 변경하세요.



In [4]:
# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/esg-survey-datasets", split="train")

# 2. OpenAI format으로 데이터 변환을 위한 함수 
def format_data(sample):
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": sample['system_prompt'],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": sample["assistant"]
            },
        ],
    }

In [5]:
# 1. 허깅페이스 허브에서 데이터셋 로드
from datasets import load_dataset
import random

dataset = load_dataset("iamjoon/esg-survey-datasets", split="train")

# 2. OpenAI format으로 데이터 변환을 위한 함수 
def format_data(sample):
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": sample['system_prompt'],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": sample["assistant"]
            },
        ],
    }

# 3. 전체 데이터에 OpenAI 포맷 전처리 적용
formatted_dataset = dataset.map(format_data)

# 4. 데이터를 7:2 비율로 학습/테스트 분할 (HuggingFace 내장 메서드 사용)
split_dataset = formatted_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

# 5. 결과 확인
print(f"전체 데이터 수: {len(formatted_dataset)}")
print(f"학습 데이터 수: {len(train_dataset)} ({len(train_dataset)/len(formatted_dataset)*100:.1f}%)")
print(f"테스트 데이터 수: {len(test_dataset)} ({len(test_dataset)/len(formatted_dataset)*100:.1f}%)")

전체 데이터 수: 4599
학습 데이터 수: 3219 (70.0%)
테스트 데이터 수: 1380 (30.0%)


In [6]:
# 6. 샘플 데이터 확인
print("\n=== 학습 데이터 샘플 ===")
for message in train_dataset[0]['messages']:
    print(f"[{message['role'].upper()}]")

    # 상위 500개 문자열만 출력
    print(message['content'][:500])
    print("-" * 50)


=== 학습 데이터 샘플 ===
[SYSTEM]
당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
삼양사 2021 지속가능경영보고서
52
임직원 역량 강화 및 포용적 조직문화 지향
ENVIRONMENTAL SOCIAL GOVERNANCE APPENDIXOVERVIEW ESG JOURNEY
인권 경영
인권 존중 경영
삼양사는 인권침해가 발생할 수 있는 분야에 관한 인권교육을 실시하여 인권 리스크를 사전에 
예방하고 임직원의 인권 역량을 높이고 있습니다. 매년 신입사원을 포함한 전 임직원을 
대상으로 직장 내 괴롭힘 교육, 성희롱 예방 교육 및 장애인 인식개선 교육을 온라인으로 
실시합니다. 또한 회사내 근로자의 고충을 청취하고 이를 처리하기 위하여 고충처리 프로세스를 
운영하고 있습니다. 이를 위하여 본사에 노사 각 1명씩의 고충처리위원을 두고 있으며, 
고충처리위원은 고충사항의
--------------------------------------------------
[USER]
질문: C3-2-2. 지배구조 요소와 관련하여 아래 항목 중 현재 재직하고 있는 회사에 해당되는 응답을 선택해주세요._1)(상장기업만 응답) 공정공시제도를 준수하고 있는가?

보기:
['1) 전혀그렇지않다', '2) 약간그렇지않다', '3) 보통이다', '4) 약간그렇다', '5) 매우그렇다']
--------------------------------------------------
[ASSISTANT]
답변: 알 수 없음
--------------------------------------------------


In [7]:
print("\n=== 테스트 데이터 샘플 ===")
for message in test_dataset[0]['messages']:
    print(f"[{message['role'].upper()}]")

    # 상위 500개 문자열만 출력
    print(message['content'][:500])
    print("-" * 50)


=== 테스트 데이터 샘플 ===
[SYSTEM]
당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
지배구조
DAESANG 2024
Sustainability Report
•지배구조 정보보호 윤리 및 준법경영리스크 관리
이사회 중심의 지배구조는 기업의 투명성과 효율성을 증진시키며, 이해관계자들과 신뢰를 구축하는 데 중요합니다. 이를 위해 대상㈜는 이사회 독립성 및 다양성 정책을   
제정하였으며, 이사회 내 과반수 이상을 사외이사로 구성하였습니다. 이사의 선임에는 전문성을 적극적으로 고려하여 재무, 공정거래, 식품안전 등 여러 분야 전문가를 선임
하였고, 다양한 관점에서 경영에 대한 조언 및 견제가 가능하도록 하고 있습니다. 당사는 앞으로도 이사회의 투명하고 책임있는 경영을 통해 이해관계자의 권익 보호와 기업
의 지속가능한 성장을 위해 노력하겠습니다.
이사회 독립성 및 다양성
대
--------------------------------------------------
[USER]
질문: C3-1-1. 지배구조 요소와 관련하여 아래 항목 중 현재 재직하고 있는 회사에 해당되는 응답을 선택해주세요._6)자금 집행과 관리 주체를 분리하여 체계적으로 운영하고 있는가?

보기:
['1) 전혀그렇지않다', '2) 약간그렇지않다', '3) 보통이다', '4) 약간그렇다', '5) 매우그렇다']
--------------------------------------------------
[ASSISTANT]
답변: 알 수 없음
--------------------------------------------------


In [7]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [8]:
# 데이터셋 저장
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1380 [00:00<?, ? examples/s]

## 2. 모델 로드 및 템플릿 적용


In [8]:
# 허깅페이스 모델 ID
model_id = "NCSOFT/Llama-VARCO-8B-Instruct" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
삼양사 2021 지속가능경영보고서
52
임직원 역량 강화 및 포용적 조직문화 지향
ENVIRONMENTAL SOCIAL GOVERNANCE APPENDIXOVERVIEW ESG JOURNEY
인권 경영
인권 존중 경영
삼양사는 인권침해가 발생할 수 있는 분야에 관한 인권교육을 실시하여 인권 리스크를 사전에 
예방하고 임직원의 인권 역량을 높이고 있습니다. 매년 신입사원을 포함한 전 임직원을 
대상으로 직장 내 괴롭힘 교육, 성희롱 예방 교육 및 장애인 인식개선 교육을 온라인으로 
실시합니다. 또한 회사내 근로자의 고충을 청취하고 이를 처리하기 위하여 고충처리 프로세스를 
운영하고 있습니다. 이를 위하여 본사에 노사 각 1명씩의 고충처리위원을 두고 있으며, 
고충처리위원은 고충사항의 접수 및 처리에 관한 대장을 작성, 비치하고 이를 1년간 보존하고 
있습니다. 또한, 보고기간내 보고된 차별 사례 및 이에 대한 시정조치는 없습니다.
상생의 노사관계
삼양사는 국제노동기구 협약 및 국내 노동법에 의한 결사의 및 단체교섭의 자유를 존중합니다. 
삼양사는 노사협의회 규정을 제정하여 조직문화 개선 활동, 고충처리 활동 등을 진행하고 
있습니다. 노사협의회를 통해 결정된 사안은 모든 임직원에게 동일하게 적용되고 있으며, 일부 
개별사안에 대해서는 대상자에게 적용됩니다. 이러한 노사협의회는 분기별로 개최됩니다. 
또한, 삼양사는 노사협의회 외 노동조합이 설립되어 운영되고 있습니다. 생산직 뿐만 아니라 
사무관리·영업·연구원들 또한 노동조합을 설립하여 사무직군의 권리를 대변하고, 노사상생 
모델을 모색합니다. 
고충처리 프로세스
고충 유형별 고충 건수, 처리율
구분
2

## 3. LoRA와 SFTConfig 설정


In [11]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

- lora_alpha: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.
- lora_dropout: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. 0.1로 설정하면 학습 중 10%의 뉴런이 비활성화.
- r: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.
- bias: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. "none"으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. "all" 또는 "lora_only"와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.
- target_modules: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 "q_proj"와 "v_proj"를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.
- task_type: LoRA가 적용되는 작업 유형을 지정합니다. "CAUSAL_LM"은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 "SEQ2SEQ_LM"(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [12]:
args = SFTConfig(
    output_dir="llama3-8b-esg-survey-model",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=2,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=4,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

- `output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

- `num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니.

- `per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있니다.

- `gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 `2`로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 용합니다.

- `gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질수 있습니다.

- `optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적기를 사용합니다.

- `logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마 로그를 기록합니다.

- `save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스마다 모델이 저장됩니다.

- `save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50스텝마다 모델을 저장합니다.

- `bf16`: `bfloat16` 정밀도를 사용하도록 설정합니다. `bfloat16`은 FP32와 유사한 범위를 제공하면서 모리와 계산 효율성을 높입니다.

- `learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 용하여 안정적인 학습을 유도합니다.

- `max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 정하여 폭발적 그래디언트를 방지합니다.

- `warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다 학습의 안정성을 높이기 위해 사용됩니다.

- `lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"costant"`는 학습률을 일정하게 유지합니다.

- `push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니. `False`로 설정하면 업로드하지 않습니다.

- `remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다.`True`로 설정하면 메모리를 절약할 수 있습니다.

- `dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset True`로 설정하여 데이터셋 준비 단계를 건너뜁니다.

- `report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn


In [13]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        messages = example["messages"]

        # LLaMA 3 채팅 템플릿 적용 (시작 토큰 포함)
        prompt = "<|begin_of_text|>"
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"

        # 마지막 assistant 메시지는 응답으로 간주하고 레이블에 포함
        text = prompt.strip()

        # 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        # assistant 응답의 시작 위치 찾기
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        # 레이블 범위 지정
        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j]  # <|eot_id|> 토큰도 포함
                break
            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 처리
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i])
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

- 라마 챗 템플릿

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations.<|eot_id|><|start_header_id|>user<|end_header_id|>

What can you help me with?<|eot_id|><|start_header_id|>assistant<|end_header_id|>|>

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 assistant 답변 부분을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

최종적으로, 배치 내 모든 샘플의 길이를 동일하게 맞추기 위해 패딩 작업을 수행합니다. 이 과정에서 입력 데이터에는 패딩 토큰 ID를 추가하고, 어텐션 마스크에는 0을 추가하며, 레이블에는 -100을 추가합니다. 모든 데이터는 PyTorch 텐서로 변환되어 반환됩니다.

In [14]:
# 최대 길이
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 7938])
어텐션 마스크 형태: torch.Size([1, 7938])
레이블 형태: torch.Size([1, 7938])


In [15]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 56773, 125441, 109760, 19954, 122115, 115036, 99901, 11, 109760, 11, 64432, 106647, 82818, 120378, 43139, 111964, 110513, 109670, 627, 109070, 78326, 99901, 18918, 82818, 120378, 43139, 111964, 48936, 29833, 108838, 50152, 102772, 364, 111012, 29833, 127409, 6, 110917, 103607, 34609, 51402, 382, 109070, 78326, 99901, 512, 27, 6190, 220, 15, 397, 11192, 111690, 101927, 56154, 220, 2366, 16, 67890, 102130, 116669, 66406, 101090, 115202, 27796, 198, 4103, 198, 94801, 102436, 55421, 103135, 104690, 102258, 57390, 101824, 99969, 27797, 82068, 66610, 102436, 113626, 67890, 104762, 198, 31460, 60939, 984, 98180, 12890, 87470, 8606, 18395, 4794, 5511, 50205, 21709, 469, 7939, 622, 6934, 75231, 198, 32428, 103131, 44215, 101090, 198, 32428, 103131, 109584, 101711, 44215, 101090, 198, 111690, 101927, 117396, 59777, 103131, 108308, 34983, 20565, 113610, 48936, 29833, 65621, 127290, 19954, 115825, 59777, 103131, 107434, 183

In [16]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
삼양사 2021 지속가능경영보고서
52
임직원 역량 강화 및 포용적 조직문화 지향
ENVIRONMENTAL SOCIAL GOVERNANCE APPENDIXOVERVIEW ESG JOURNEY
인권 경영
인권 존중 경영
삼양사는 인권침해가 발생할 수 있는 분야에 관한 인권교육을 실시하여 인권 리스크를 사전에 
예방하고 임직원의 인권 역량을 높이고 있습니다. 매년 신입사원을 포함한 전 임직원을 
대상으로 직장 내 괴롭힘 교육, 성희롱 예방 교육 및 장애인 인식개선 교육을 온라인으로 
실시합니다. 또한 회사내 근로자의 고충을 청취하고 이를 처리하기 위하여 고충처리 프로세스를 
운영하고 있습니다. 이를 위하여 본사에 노사 각 1명씩의 고충처리위원을 두고 있으며, 
고충처리위원은 고충사항의 접수 및 처리에 관한 대장을 작성, 비치하고 이를 1년간 보존하고 
있습니다. 또한, 보고기간내 보고된 차별 사례 및 이에 대한 시정조치는 없습니다.
상생의 노사관계
삼양사는 국제노동기구 협약 및 국내 노동법에 의한 결사의 및 단체교섭의 자유를 존중합니다. 
삼양사는 노사협의회 규정을 제정하여 조직문화 개선 활동, 고충처리 활동 등을 진행하고 
있습니다. 노사협의회를 통해 결정된 사안은 모든 임직원에게 동일하게 적용되고 있으며, 일부 
개별사안에 대해서는 대상자에게 적용됩니다. 이러한 노사협의회는 분기별로 개최됩니다. 
또한, 삼양사는 노사협의회 외 노동조합이 설립되어 운영되고 있습니다. 생산직 뿐만 아니라 
사무관리·영업·연구원들 또한 노동조합을 설립하여 사무직군의 권리를 대변하고, 노사상생 
모델을 모색합니다. 
고충처리 프로세스
고충 유

In [17]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [18]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
답변: 알 수 없음<|eot_id|>


### input_ids와 labels는 어떻게 생성되는가?


LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 이를 예시 문장과 정수 인코딩을 통해 상세히 설명하겠습니다.

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다.
사용자가 `안녕하세요, 오늘 날씨는 어떤가요?`라고 물었고,
모델은 `안녕하세요! 오늘 날씨는 맑고 화창합니다.`라고 응답해야 한다고 합시다.

LLaMA 3에서는 다음과 같은 템플릿 구조를 사용합니다:

`<|begin_of_text|><|start_header_id|>user<|end_header_id|>
안녕하세요, 오늘 날씨는 어떤가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
안녕하세요! 오늘 날씨는 맑고 화창합니다.<|eot_id|>`

이 전체 텍스트는 토크나이저에 의해 정수 시퀀스로 변환됩니다. 예시로 단순화된 정수 시퀀스는 다음과 같다고 가정합니다:

`input_ids = [1001, 2001, 3001, 4001, 5001, 6001, 7001, 1002, 1001, 8001, 9001, 1003, 2002]`

여기서 모델이 예측해야 할 영역은 assistant의 응답 부분인
`안녕하세요! 오늘 날씨는 맑고 화창합니다.`에 해당하는 토큰들입니다.
따라서 `labels`는 다음과 같이 설정됩니다:

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, 8001, 9001, 1003, 2002]`

이처럼 `labels`는 모델의 출력이 필요한 영역만을 포함하고, 나머지 부분은 `-100`으로 채워져
모델이 실제로 예측하고 오차를 계산해야 하는 대상(학습 대상)에서 제외됩니다.

이를 통해 모델은 불필요한 입력 부분을 학습하지 않고, assnt 응답 부분에만 집중할 수 있습니다.
"""


## 5. 학습

In [19]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [20]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,2.103700
20,0.324500
30,0.133500
40,0.110300
50,0.158700
60,0.110400
70,0.131900
80,0.119700
90,0.110000
100,0.154700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

## 6. 테스트 데이터 준비


실제 모델에 입력을 넣을 때에는 입력의 뒤에 '<|start_header_id|>assistant<|end_header_id|>\n'가 부착되어서 넣는 것이 좋습니다. 그래야만 모델이 바로 답변을 생성합니다.

In [9]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [16]:
print(prompt_lst[500])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
필요 시 외부전문가와 협력하여 운영 과정의 공정성 및 투명성 확보
•
 202
1년 총 6건의 고충이 접수되었고 피해자의 요구를 토대로 인사
위원회 등 내부규정에 따라 100% 처리 완료
접수 면담 사실관계 조사 해결
구분 단위 2019 2020 2021
총 고충건수 건 5 4 6
고충 처리 건수 건 5 4 6
고충 처리율 % 100 100 100
직책자에 대해 연 2회 이상 교육 수료라는 
강화된 내부규정을 적용하여 윤리강령 숙
지가 필수적으로 이뤄지도록 하고 있음
성희롱 예방교육
직장 내 괴롭힘 예방을 통해 건강하고 행복한 
조직문화를 함께 만들어 나가고자 직책자와 
비직책자 모두에게 교육을 실시하고 있음
직장내 괴롭힘 예방교육
거래업체와의 공정하고 균형 있는 발전을 위해 공정거래 관련 사항은 별도 교육을 편성하여 
해당 직무자와 직책자에게 제공하고 있음
공정거래, 하도급 윤리 및 부패방지 교육
장애인에 대한 직장 내 편견을 제거함으로
써 밝고 건강한 직장문화 조성을 위해 연 1
회 전사원에게 교육을 실시하고 있음
장애인 인식개선 교육
자유경쟁시장 질서를 존중하고 윤리적 기업
문화를 형성하기 위해 신입사원, 예비리더, 
신임직책자, 임원을 대상으로 교육 실시함
윤리경영 교육
허용가능 여부
---
</document 0>

<document 1>
---
보고서 개요
대상주식회사(이하 대상(주))는 지속가능경영 활동 및 성과를 다
양한 이해관계자들에게 공유하고자 매년 지속가능경영보고서를 
발간하고 있으며, 가장 최근 보고서는 2021년 11월에 발간하였습
니다. 본 보고서를 통해 ESG 분류체계에 따른 환경, 사회, 지배구
조 관련 정보를 투명하게 공

In [19]:
print(label_lst[500])


답변: 알 수 없음


## 7. 파인튜닝 모델 테스트

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [12]:
peft_model_id = "llama3-8b-esg-survey-model/checkpoint-800"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [13]:
eos_token = tokenizer("<|eot_id|>", add_special_tokens=False)["input_ids"][0]

In [14]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [20]:
for prompt, label in zip(prompt_lst[700:703], label_lst[700:703]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label.replace('Answer: ', '')}") # 데이터에서 일부 'Answer: '가 잘리지 않은 경우를 대비
    print("-"*50)

    response:
답변: 5) 매우 중요하다
    label:

답변: 5) 매우 중요하다
--------------------------------------------------
    response:
답변: 5) 매우그렇다
    label:

답변: 5) 매우그렇다
--------------------------------------------------
    response:
답변: 5) 매우그렇다
    label:

답변: 알 수 없음
--------------------------------------------------
